# DATA 602 - FINAL PROJECT
## CRAFT BREWERY GROWTH & STATE EMPLOYMENT

### IMPORT MODULES & PACKAGES  

This project was used as an opportunity to hone a few Python skills that I wanted to sharpen - mainly web scraping and working with regular expressions, opening files from Excel, and using a different visualization package (`Bokeh`).  

In [1]:
from bs4 import BeautifulSoup
import urllib2
import json
import requests
import re
import pandas as pd
import openpyxl
from bokeh.charts import Bar, Scatter, output_file, show
from bokeh.layouts import row, gridplot
from bokeh.io import output_notebook

pd.options.display.max_columns = 35

### SCRAPE BEER DATA

Most of the data that I wanted to use is located on the National Brewers Association website. One of the reasons I chose this data source was the line charts for each state - they are javascript, and I wanted to see if I could pull the information out, in addition to the information stored in more conventional html format.  

In [2]:
webpage = ('https://www.brewersassociation.org/statistics/by-state/')
html = urllib2.urlopen(webpage).read()

In [3]:
soup = BeautifulSoup(html, 'lxml')

Here we're just grabbing the names of the states, basically a BeautifulSoup practice run. After that, the rest of the data is stored in the `span` tag, which is simply accessed from simple list indexing.  

In [4]:
state_html = soup.find_all('h1')
state_html = state_html[2:]
state_html[0:5]

states = []
for st in state_html:
    states.append(str(st.get_text()))

states[0:5]

['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California']

In [5]:
stats = soup.find_all('span', {'class':'count'}, text=True)

craft_breweries = stats[::6]
per_capita = stats[1::6]
econ_impact = stats[2::6]
impact_capita = stats[3::6]
barrels = stats[4::6]
gallons_per = stats[5::6]

#### GET TEXT, CONVERT TO NUMERICAL FORMAT  

Since all of the numerical data is stored as text, and there were multiple lists of values, I created a quick function to do the repetitive work of getting the text, replacing the commas, then converting the text to a float value. The results were then stored in an object of the same name (i.e. `barrels = get_text(barrels`).  

In [6]:
# create lists, append each line to list, create dataframe
def get_text(tag_list):
    """Converts html text to string, removes any commas, and coverts to float.
    Args:
        tag_list(list): A list of html tags
    
    Returns:
        temp(list): A list of float values from converted html text.    
    """
    temp = []
    for item in tag_list:
        temp_str = item.get_text()
        temp_str = temp_str.replace(',', '')
        temp.append(float(temp_str))
    return temp

In [8]:
craft_breweries = get_text(craft_breweries)
per_capita = get_text(per_capita)
econ_impact = get_text(econ_impact)
impact_capita = get_text(impact_capita)
barrels = get_text(barrels)
gallons_per = get_text(gallons_per)


In [9]:
# First five records, and length of list for demonstration purposes

gallons_per[:5], len(gallons_per)

([0.4, 12.7, 1.2, 0.4, 4.3], 51)

### Getting Javascript Values

There's probably a more pythonic way to do this, but I was happy with the results. Gathering the number of breweries per year for each state was a little more complicated.  

BeautifulSoup was used to access the javascript, which was then converted to a string, split up on the `<script>` tag, and then a "for-loop" with the help of some regular expressions filtered out the rest.  

In [10]:
state_stats = soup.find('section', {'id': 'primary'})
script = state_stats.findAll('script')

script = str(script)

script = script.split('<script>')
values = script[2]

brew_per_yr = []
for i in range(1, len(script)):
    date = re.findall(r'"value": "\d{1,3}"', script[i])
    for j in date:
        #print j[9:14]
        hi = re.search(r'\d{1,3}', j)
        brew_per_yr.append(hi.group())
        
brew_per_yr[0:10]

['6', '10', '13', '19', '24', '20', '22', '22', '22', '27']

Similar to the other data above, list indices were used to access the information stored in a list in the appropriate year.  

In [11]:
brew_2011 = brew_per_yr[::5]
brew_2012 = brew_per_yr[1::5]
brew_2013 = brew_per_yr[2::5]
brew_2014 = brew_per_yr[3::5]
brew_2015 = brew_per_yr[4::5]
brew_2015[0:5]

['24', '27', '78', '26', '518']

### Create Beer Dataframe

Each list of the data was combined using the `zip` function, columns re-named, and then the values of the `State` column were renamed for later merging with the other DataFrames that will be created. It was easier to change all of the full state names to abbreviations for one data set, rather than the other way around for two others.  

In [12]:
beer_table = zip(states, craft_breweries, per_capita, econ_impact, impact_capita, barrels, 
                 gallons_per, brew_2011, brew_2012, brew_2013, brew_2014, brew_2015)

beer_df = pd.DataFrame(beer_table, columns = ['State', 'Craft_Breweries', 'Breweries Per Capita', 'Econ. Impact', 
                                              'Econ.Impact/Capita', 'Barrels', 'Gallons/Adult', 
                                              '2011', '2012', '2013', '2014', '2015'])

beer_df['State'] = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL',  # use state abbrevs for later join
 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 
 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 
 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 
 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

beer_df.head()

,State,Craft_Breweries,Breweries Per Capita,Econ. Impact,Econ.Impact/Capita,Barrels,Gallons/Adult,2011,2012,2013,2014,2015
0,AL,24.0,0.7,438.0,125.53,50369.0,0.4,6,10,13,19,24
1,AK,27.0,5.3,239.0,469.09,209644.0,12.7,20,22,22,22,27
2,AZ,78.0,1.6,960.0,203.25,187213.0,1.2,34,45,47,53,78
3,AR,26.0,1.2,324.0,153.03,24623.0,0.4,6,10,13,19,26
4,CA,518.0,1.9,6887.0,251.89,3799785.0,4.3,270,319,381,431,518


### Downloaded Beer Data from TTB

This next set of data was obtained from the TTB website (www.ttb.gov), the Alcohol and Tobacco Tax and Trade Bureau. They do not supply a nice API or even .csv files, just Excel file formats or .pdf files containing state-by-state beer production stats.  

Since most of the data I have worked with in Python is conveniently in .csv files, I took the opportunity to learn a bit about using `openpyxl` to access some of this data. Unfortunately, storing the information on GitHub and pulling it directly from there wasn't an option, so you will need to download and store the files in the same directory as this notebook file.  

In [13]:
wb = openpyxl.load_workbook('aggr-data-beer-2010-2015.xlsx')

/Users/Logan/anaconda/lib/python2.7/site-packages/openpyxl/workbook/names/named_range.py:124: UserWarning: Discarded range with reserved name
  warnings.warn("Discarded range with reserved name")


It's a fairly simple method to access the information. `openpyxl` can access sheets by name, and columns/rows as well - sort of like "slicing" a big Python list.  

The individual sheets were stored in objects, and then nested for-loops were used to get the information from each column and store it in a list object.

In [14]:
wb.get_sheet_names()

[u'Tax Determined (Premise Use)', u'Bottles and Cans', u'Barrels and Kegs']

In [15]:
bottle_can = wb.get_sheet_by_name('Bottles and Cans')
barrel_keg = wb.get_sheet_by_name('Barrels and Kegs')

bc_list = []
bk_list = []

for i in range(len(bottle_can.columns)):
    cols = []
    b = bottle_can.columns[i]
    for bc in b:
        cols.append(bc.value)
    bc_list.append(cols)

for i in range(len(barrel_keg.columns)):
    cols = []
    b = barrel_keg.columns[i]
    for bk in b:
        cols.append(bk.value)
    bk_list.append(cols)

bc_list[1][2], bk_list[1][2]

(80889.21, 54395.32)

#### CREATE TTB DATAFRAME  

Same was used to put together all of the individual lists from the Bottles & Cans sheet, and the Barrels & Kegs sheet, then columns were re-named, then empty and formula rows were dropped from the DataFrame.  

The two pandas DataFrames were then merged on the `State` column, and the columns were re-ordered by name. A preview of the findal combined DataFrame is below.  

In [16]:
# Beer Production by State - Bottle & Cans (in Barrels)
tbb_bc = pd.DataFrame(zip(bc_list[0], bc_list[1], bc_list[2], bc_list[3], bc_list[4], bc_list[5], bc_list[6]))

tbb_bc.columns =  (['State', 'Btls & Cans (in Barrels) 2010', 'Btls & Cans (in Barrels) 2011', 
                    'Btls & Cans (in Barrels) 2012', 'Btls & Cans (in Barrels) 2013', 'Btls & Cans (in Barrels) 2014', 
                    'Btls & Cans (in Barrels) 2015'])

tbb_bc = tbb_bc.reindex(tbb_bc.index.drop([0, 1, 53, 54]))

In [17]:
# Beer Production by State - Barrels & Kegs (in Barrels)
tbb_bk = pd.DataFrame(zip(bk_list[0], bk_list[1], bk_list[2], bk_list[3], bk_list[4], bk_list[5], bk_list[6]))

tbb_bk.columns =  (['State', 'Barrels & Kegs (in Barrels) 2010', 'Barrels & Kegs (in Barrels) 2011', 
                    'Barrels & Kegs (in Barrels) 2012', 'Barrels & Kegs (in Barrels) 2013', 
                    'Barrels & Kegs (in Barrels) 2014', 'Barrels & Kegs (in Barrels) 2015'])

tbb_bk = tbb_bk.reindex(tbb_bk.index.drop([0, 1, 53, 54]))

In [18]:
tbb = pd.merge(tbb_bc, tbb_bk, on='State')

tbb = tbb[['State', 'Barrels & Kegs (in Barrels) 2010', 'Btls & Cans (in Barrels) 2010', 
           'Barrels & Kegs (in Barrels) 2011', 'Btls & Cans (in Barrels) 2011', 
           'Barrels & Kegs (in Barrels) 2012', 'Btls & Cans (in Barrels) 2012', 
           'Barrels & Kegs (in Barrels) 2013', 'Btls & Cans (in Barrels) 2013', 
           'Barrels & Kegs (in Barrels) 2014', 'Btls & Cans (in Barrels) 2014', 
           'Barrels & Kegs (in Barrels) 2015', 'Btls & Cans (in Barrels) 2015',]]

tbb.head()

,State,Barrels & Kegs (in Barrels) 2010,Btls & Cans (in Barrels) 2010,Barrels & Kegs (in Barrels) 2011,Btls & Cans (in Barrels) 2011,Barrels & Kegs (in Barrels) 2012,Btls & Cans (in Barrels) 2012,Barrels & Kegs (in Barrels) 2013,Btls & Cans (in Barrels) 2013,Barrels & Kegs (in Barrels) 2014,Btls & Cans (in Barrels) 2014,Barrels & Kegs (in Barrels) 2015,Btls & Cans (in Barrels) 2015
0,AK,54395.3,80889.2,58519.7,94781.6,65679.1,105416,68867.6,112009,73823.6,129182,74606.3,126309
1,AL,547.69,0,1810.06,658.96,6453.45,2485.79,11747.7,6634.43,13750,9521.26,11465.2,10543
2,AR,1318.17,1445.82,1144.17,1382.06,125.94,8,1078.21,8,2787.48,800.09,6965.83,872.11
3,AZ,24410.5,10380.3,32848.9,14057.6,44792.7,18411.2,59074,22318.7,80134.1,29645,74997.9,44081.1
4,CA,2.19855e+06,1.90956e+07,2.16603e+06,1.82045e+07,2.27266e+06,1.89403e+07,2.38332e+06,1.78355e+07,2.32851e+06,1.80529e+07,2.17332e+06,1.76706e+07


## BLS DATA  

Originally, I intended to obtain this data using the public ("V1") API made available by the Bureau of Labor Statistics. However, there appears to be a problem with the API calls, this may have something to do with the BLS and Census website changing the URLs and locations of some of their data only days ago.  

Instead, I simply relied on BeautifulSoup to get the data I was looking for. State-by-state unemployment rate averages for the entire year are not easy to find on the BLS website. In the end, it was probably better to access this information this way, rather than using the API.  

In [19]:
def get_bls(url):
    """Obtains unemployment rank, state name, and unemployment rate for a given URL
    on the bls.gov website.
    
    Args: 
        url(string): the website address for the table data given as a string.  
        
    Returns: 
        a list of lists containing the rank, state, and unemployment rate data.
    """
    webpage = (url)
    html = urllib2.urlopen(webpage).read()
    soup = BeautifulSoup(html, 'lxml')
    
    bls = soup.find_all('td', text=True)
    rank = bls[0::3]
    state = bls[1::3]
    rate = bls[2::3]
    
    ranks = []
    for i in rank:
        ranks.append((i.get_text()))
    
    states = []
    for i in state:
        states.append(i.get_text())
    
    rates = []
    for i in rate:
        rates.append(float(i.get_text()))
    
    return [ranks, states, rates]

A quick function to create individual DataFrames and rename the columns (to differentiate after combining the data) was created.  

In [20]:
def df_bls(bls_list, rank_yr, rate_yr):
    """Takes lists created from BLS data, creates a DataFrame, and renames the columns.
    
    Args:
        bls_list(list): a list of lists generated by the get_bls function.
        rank_yr(string): a string to name the rank column and designate the year of the data.
        rate_yr(string): a string to name the rate column and designate the year of the data.
        
    Returns:
        bls_unemp(pandas DataFrame): A pandas DataFrame of the BLS data.    
    """
    bls_unemp = pd.DataFrame(zip(bls_list[0], bls_list[1], bls_list[2]), 
                             columns=['{}'.format(rank_yr), 'State', '{}'.format(rate_yr)])
    return bls_unemp

Lastly, each URL of the location for each year of unemployment data is passed to the `get_bls` function, then passed to the `df_bls` function and stored in an appropriately named Python object:  

In [21]:
unemp_15 = get_bls('https://www.bls.gov/lau/lastrk15.htm')
unemp_15 = df_bls(unemp_15, '2015 Rank', '2015 Rate')

unemp_14 = get_bls('https://www.bls.gov/lau/lastrk14.htm')
unemp_14 = df_bls(unemp_14, '2014 Rank', '2014 Rate')

unemp_13 = get_bls('https://www.bls.gov/lau/lastrk13.htm')
unemp_13 = df_bls(unemp_13, '2013 Rank', '2013 Rate')

unemp_12 = get_bls('https://www.bls.gov/lau/lastrk12.htm')
unemp_12 = df_bls(unemp_12, '2012 Rank', '2012 Rate')

unemp_11 = get_bls('https://www.bls.gov/lau/lastrk11.htm')
unemp_11 = df_bls(unemp_11, '2011 Rank', '2011 Rate')

A "chained" merge is used to combine all of the individual unemployment data sets, the columns are re-ordered, and the state names are changed again to abbreviations.  A preview of the first five rows of the BLS unemployment data frame is below.  

In [22]:
# df1.merge(df2,on='name').merge(df3,on='name')
state_unemp = unemp_15.merge(unemp_14, 
                             on='State').merge(unemp_13, 
                                               on='State').merge(unemp_12, 
                                                                 on='State').merge(unemp_11, 
                                                                                   on='State')

state_unemp = state_unemp[['State', '2011 Rank', '2011 Rate', '2012 Rank', '2012 Rate', 
                           '2013 Rank', '2013 Rate', '2014 Rank', '2014 Rate', 
                           '2015 Rank', '2015 Rate']]  # Reorder columns

state_unemp = state_unemp.drop(state_unemp.index[[0]])

#state_unemp = state_unemp.set_value(51, 'State', 'DC')

state_unemp['State'] = ['ND', 'NE', 'SD', 'NH', 'UT', 'HI', 'IA', 'MN', 'VT', 'CO',  # use state abbrevs for later join
 'ID', 'MT', 'KS', 'OK', 'WY', 'ME', 'VA', 'TX', 'WI', 'IN', 
 'DE', 'OH', 'MA', 'MO', 'PA', 'AR', 'MD', 'NY', 'FL', 'KY', 
 'MI', 'CT', 'NJ', 'NC', 'OR', 'WA', 'TN', 'GA', 'IL', 'RI', 
 'SC', 'AL', 'AZ', 'CA', 'LA', 'AK', 'MS', 'NM', 'NV', 'WV', 'DC']

state_unemp.head()

,State,2011 Rank,2011 Rate,2012 Rank,2012 Rate,2013 Rank,2013 Rate,2014 Rank,2014 Rate,2015 Rank,2015 Rate
1,ND,1,3.5,1,3.1,1,2.9,1,2.7,1,2.7
2,NE,2,4.4,2,4.0,2,3.8,2,3.3,2,3.0
3,SD,3,4.7,3,4.3,2,3.8,3,3.4,3,3.1
4,NH,4,5.4,9,5.5,10,5.1,9,4.3,4,3.4
5,UT,12,6.7,8,5.4,5,4.6,4,3.8,5,3.5


### CREATE MASTER DATAFRAME

The three data frames (`beer_df`, `tbb`, and `state_unemp`) are the merged together. Also, the data in many of the columns was captured as a pandas "object" (text), and needed to be converted into a float value in order to do any calculations.  

A preview of the final DataFrame is below.  

In [23]:
master_df = beer_df.merge(tbb, on='State').merge(state_unemp, on='State')

for col in ['2011', '2012', '2013', '2014', '2015', 'Barrels & Kegs (in Barrels) 2010',  
            'Btls & Cans (in Barrels) 2010', 'Barrels & Kegs (in Barrels) 2011', 
            'Btls & Cans (in Barrels) 2011', 'Barrels & Kegs (in Barrels) 2012', 
            'Btls & Cans (in Barrels) 2012', 'Barrels & Kegs (in Barrels) 2013', 
            'Btls & Cans (in Barrels) 2013', 'Barrels & Kegs (in Barrels) 2014', 
            'Btls & Cans (in Barrels) 2014', 'Barrels & Kegs (in Barrels) 2015', 
            'Btls & Cans (in Barrels) 2015', '2011 Rank', '2012 Rank', '2013 Rank', 
            '2014 Rank', '2015 Rank']:
    master_df[col] = master_df[col].astype('float64')  # need to convert column types from object to float64

master_df

,State,Craft_Breweries,Breweries Per Capita,Econ. Impact,Econ.Impact/Capita,Barrels,Gallons/Adult,2011,2012,2013,2014,2015,Barrels & Kegs (in Barrels) 2010,Btls & Cans (in Barrels) 2010,Barrels & Kegs (in Barrels) 2011,Btls & Cans (in Barrels) 2011,Barrels & Kegs (in Barrels) 2012,Btls & Cans (in Barrels) 2012,Barrels & Kegs (in Barrels) 2013,Btls & Cans (in Barrels) 2013,Barrels & Kegs (in Barrels) 2014,Btls & Cans (in Barrels) 2014,Barrels & Kegs (in Barrels) 2015,Btls & Cans (in Barrels) 2015,2011 Rank,2011 Rate,2012 Rank,2012 Rate,2013 Rank,2013 Rate,2014 Rank,2014 Rate,2015 Rank,2015 Rate
0,AL,24.0,0.7,438.0,125.53,50369.0,0.4,6.0,10.0,13.0,19.0,24.0,547.69,0.00,1810.06,658.96,6453.45,2485.79,11747.74,6634.43,13749.96,9521.26,11465.23,10543.00,40.0,9.6,32.0,8.0,29.0,7.2,40.0,6.8,42.0,6.1
1,AK,27.0,5.3,239.0,469.09,209644.0,12.7,20.0,22.0,22.0,22.0,27.0,54395.32,80889.21,58519.66,94781.57,65679.13,105416.32,68867.58,112008.59,73823.62,129181.74,74606.28,126309.22,19.0,7.6,20.0,7.1,26.0,6.9,43.0,6.9,46.0,6.5
2,AZ,78.0,1.6,960.0,203.25,187213.0,1.2,34.0,45.0,47.0,53.0,78.0,24410.52,10380.26,32848.92,14057.56,44792.72,18411.25,59074.04,22318.65,80134.06,29645.05,74997.94,44081.10,38.0,9.5,35.0,8.3,35.0,7.7,40.0,6.8,42.0,6.1
3,AR,26.0,1.2,324.0,153.03,24623.0,0.4,6.0,10.0,13.0,19.0,26.0,1318.17,1445.82,1144.17,1382.06,125.94,8.00,1078.21,8.00,2787.48,800.09,6965.83,872.11,26.0,8.3,28.0,7.6,30.0,7.3,26.0,6.1,26.0,5.2
4,CA,518.0,1.9,6887.0,251.89,3799785.0,4.3,270.0,319.0,381.0,431.0,518.0,2198550.10,19095643.27,2166028.35,18204514.85,2272663.96,18940276.02,2383322.71,17835501.70,2328511.86,18052949.81,2173320.10,17670616.67,50.0,11.7,49.0,10.4,48.0,8.9,47.0,7.5,44.0,6.2
5,CO,284.0,7.3,2716.0,709.39,1775831.0,14.1,126.0,151.0,175.0,235.0,284.0,2279812.02,18742266.67,2277201.65,18820319.55,2290799.82,19267902.29,2177094.68,18549122.13,2207491.04,18035909.80,2082879.02,17755216.77,29.0,8.4,31.0,7.9,25.0,6.8,15.0,5.0,10.0,3.9
6,CT,35.0,1.3,569.0,216.34,105484.0,1.2,16.0,21.0,23.0,27.0,35.0,6823.23,4822.66,8033.77,5578.56,6849.69,6372.31,20486.04,21549.90,33903.03,50874.70,38093.71,79904.60,31.0,8.8,35.0,8.3,38.0,7.8,36.0,6.6,32.0,5.6
7,DE,15.0,1.9,264.0,389.08,248742.0,11.2,7.0,9.0,10.0,11.0,15.0,48117.54,89753.27,57886.92,109745.27,65524.44,129493.68,75629.27,148156.85,82037.96,167806.22,83489.86,176483.90,17.0,7.5,23.0,7.2,19.0,6.7,20.0,5.7,21.0,4.9
8,DC,10.0,2.2,149.0,293.10,29727.0,1.8,6.0,6.0,9.0,8.0,10.0,0.00,184.80,1366.16,332.23,4257.53,1727.77,5934.43,3546.13,8335.80,4265.73,8925.18,9222.28,44.0,10.2,41.0,9.0,45.0,8.5,50.0,7.8,51.0,6.9
9,FL,151.0,1.0,2056.0,138.40,1207936.0,2.5,45.0,57.0,66.0,111.0,151.0,1247064.73,9232548.52,1287687.46,9380672.31,1289450.36,9738023.02,1260334.83,9521482.11,1241152.79,9438491.14,1243754.48,9841609.40,42.0,10.0,38.0,8.5,30.0,7.3,29.0,6.3,29.0,5.4


In [24]:
master_df.dtypes[0:5]

State                    object
Craft_Breweries         float64
Breweries Per Capita    float64
Econ. Impact            float64
Econ.Impact/Capita      float64
dtype: object

## CRAFT BEER STATS BY STATE  

In the sections below, some basic insights on the data gathered are displayed and visualized.  

### NUMBER CRAFT BREWERIES BY STATE

In [25]:
output_notebook()

Loading BokehJS ...

In [26]:
sort_df = master_df.ix[:, 0:2]
sort_df.sort_values(by='Craft_Breweries', ascending=False)[:10]

,State,Craft_Breweries
4,CA,518.0
47,WA,305.0
5,CO,284.0
37,OR,228.0
32,NY,208.0
22,MI,205.0
43,TX,189.0
38,PA,178.0
33,NC,161.0
13,IL,157.0


In [27]:
sort_df.sort_values(by='Craft_Breweries', ascending=False)[-10:]

,State,Craft_Breweries
18,LA,20.0
7,DE,15.0
36,OK,14.0
39,RI,14.0
41,SD,14.0
11,HI,13.0
48,WV,12.0
8,DC,10.0
34,ND,9.0
24,MS,8.0


Sorting the master DataFrame by different columns, we can get a quick look at states with the highest or lowest number of craft breweries by state. Not surprisingly, the states with the highest number of breweries also have the largest populations. Some states, like Colorado and Oregon, are not even in the top 20 states with regards to population, but have a large number of breweries.  

Near the bottom of the list, are mostly states with smaller populations, or are small in terms of square miles. Aside from population, many of these differences can be attributed to culture and state laws.  

### CRAFT BREWERIES PER CAPITA

In [28]:
sort_df = master_df.ix[:, 0:3]
sort_df.sort_values(by='Breweries Per Capita', ascending=False)[:10]

,State,Craft_Breweries,Breweries Per Capita
45,VT,44.0,9.4
37,OR,228.0,7.7
5,CO,284.0,7.3
26,MT,49.0,6.5
19,ME,59.0,5.9
47,WA,305.0,5.9
50,WY,23.0,5.5
1,AK,27.0,5.3
29,NH,44.0,4.5
12,ID,50.0,4.4


Looking at the quantity of craft beer breweries per capita, Vermont rises to the top, but only because of the extremely small population of the state (VT ranks 49 out of 50 states in terms of population). Again, we see some of the same states with large numbers of breweries regardless of population. Making a broad assumption, many of the states with high numbers of breweries per capita have a lot of outdoor culture (hiking, skiing, etc.), something you may want to keep in mind if opening your own brewery.

### CRAFT BEER CONSUMPTION

In [29]:
# Top 10 States by Beer Consumption per Adult
sort_df = master_df[['State', 'Barrels', 'Gallons/Adult']]
top_gal = sort_df.sort_values(by='Gallons/Adult', ascending=False)[:10]

bar1 = Bar(top_gal, values='Gallons/Adult', label=['State'],
           title="TOP STATES, Gal. per Adult Produced", legend=False, color='#F0B27A', plot_width=800)

bar1.title.text_font_size = '16pt'


# Top 10 States by Barrels Produced
sort_df = master_df[['State', 'Barrels', 'Gallons/Adult']]
top_barrel = sort_df.sort_values(by='Barrels', ascending=False)[:10]

bar2 = Bar(top_barrel, values='Barrels', label=['State'],
           title="TOP STATES, Barrels Produced", legend=False, color='#D68910', plot_width=800)

bar2.title.text_font_size = '16pt'


show(gridplot([[bar1, bar2]], plot_width=450, plot_height=600))  # side-by-side bar charts

The number of breweries doesn't tell the whole story. A state may have a large number of breweries, but some states are home to breweries with larger distribution networks, or higher production amounts.  

Examining the states with the most gallons per adult, we see a lot of the smaller states with higher numbers of breweries per capita. When looking at the top states for production, there are many higher population states, but these states are also home to some of the largest craft breweries (D. G. Yuengling in PA, Sierra Nevada and Lagunitas in CA, New Belgium Brewing in CO, etc.)

### TOP TEN STATES, OVERALL BEER PRODUCTION  

Below are the top ten states for overall beer production in the U.S., not just for craft beer and breweries, as all of the other tables have demonstrated.  

The totals below (for 2010 and 2015) are for totals of Barrels, Kegs, Bottles & Cans, so most of the ways in which beer is distributed is captured. The states listed are home to the larger "craft" breweries (Massachusetts, home to Sam Adams, finally makes an appearance), but many of the larger beer companies (SAB Miller, InBev, etc.) don't just exclusively make beer in the states they are known for (Missouri, Colorado), and those states make an appearance in the top 10 largest beer producing states in 2015.  

In [30]:
sort_df = master_df

sort_df['2015 Total'] = sort_df['Barrels & Kegs (in Barrels) 2015'] + sort_df['Btls & Cans (in Barrels) 2015']

sort_df[['State', 'Barrels & Kegs (in Barrels) 2015', 
         'Btls & Cans (in Barrels) 2015', '2015 Total']].sort_values(by='2015 Total', ascending=False)[:10]

,State,Barrels & Kegs (in Barrels) 2015,Btls & Cans (in Barrels) 2015,2015 Total
4,CA,2173320.10,17670616.67,19843936.77
5,CO,2082879.02,17755216.77,19838095.79
35,OH,1616581.07,18042852.11,19659433.18
43,TX,1026860.59,18138800.58,19165661.17
25,MO,881308.39,12493739.84,13375048.23
10,GA,462768.63,12868266.98,13331035.61
46,VA,1017631.05,11190922.96,12208554.01
9,FL,1243754.48,9841609.40,11085363.88
49,WI,936633.30,9089798.44,10026431.74
32,NY,588031.67,7218436.33,7806468.00


Despite large states with many breweries and large beer producting corporations listed above, the states with the largest volume change in production from 2010 to 2015 are listed below. We see states that have been mentioned previously when talking about craft beer production (PA, OH, MI, etc.), but these states are home to some larger craft brewers, as well as some of the "craft" brands owned by the larger mass-production breweries (like Blue Moon by Miller Coors, and Shock Top by Anheuser-Busch).  

In [31]:
sort_df = master_df

sort_df['2010 Total'] = sort_df['Barrels & Kegs (in Barrels) 2010'] + sort_df['Btls & Cans (in Barrels) 2010']
sort_df['2015 Total'] = sort_df['Barrels & Kegs (in Barrels) 2015'] + sort_df['Btls & Cans (in Barrels) 2015']
sort_df['5-Year Production Change'] = sort_df['2015 Total'] - sort_df['2010 Total']

prod_change = sort_df[['State', '2010 Total', '2015 Total',
                       '5-Year Production Change']].sort_values(by='5-Year Production Change', ascending=False)[:10]

prod_change

,State,2010 Total,2015 Total,5-Year Production Change
38,PA,2688677.44,5297515.74,2608838.30
35,OH,18275684.57,19659433.18,1383748.61
22,MI,247959.16,871244.89,623285.73
9,FL,10479613.25,11085363.88,605750.63
13,IL,998286.75,1391398.22,393111.47
23,MN,291980.47,467888.42,175907.95
43,TX,19014604.55,19165661.17,151056.62
7,DE,137870.81,259973.76,122102.95
21,MA,170474.01,287977.80,117503.79
19,ME,146647.54,254138.25,107490.71


### CRAFT BEER AS A PERCENT OF OVERALL PRODUCTION

In the table created below, craft beer is a large share of overall production in five states (35%+), and then quickly drops off from there to more modest shares of overall beer production.  

New Jersey is home to many craft breweries, but also one plant for Anheuser-Busch in Newark, which producers Budweiser and Rolling Rock in the state. Since this is the only large brewer, but it is a state with a high and dense population, and the demographics and economy support a significant number of craft breweries.  

A similar story exists in the other states listed - large numbers of craft beer breweries, with an existing "non-craft" production facility located somewhere in the state.  

In [32]:
sort_df = master_df

sort_df['2015 Total'] = sort_df['Barrels & Kegs (in Barrels) 2015'] + sort_df['Btls & Cans (in Barrels) 2015']
sort_df['Craft % of Total Beer Prod.'] = sort_df['2015 Total'] / sort_df['Barrels']

craft_pct = sort_df[['State', 'Barrels & Kegs (in Barrels) 2015', 
                     'Btls & Cans (in Barrels) 2015', '2015 Total', 
                     'Barrels', 'Craft % of Total Beer Prod.']].sort_values(by='Craft % of Total Beer Prod.', ascending=False)[:10]

craft_pct

,State,Barrels & Kegs (in Barrels) 2015,Btls & Cans (in Barrels) 2015,2015 Total,Barrels,Craft % of Total Beer Prod.
30,NJ,422379.27,3219516.57,3641895.84,79942.0,45.556727
46,VA,1017631.05,11190922.96,12208554.01,274111.0,44.538723
25,MO,881308.39,12493739.84,13375048.23,359864.0,37.166953
10,GA,462768.63,12868266.98,13331035.61,365015.0,36.521884
29,NH,347827.51,2856383.87,3204211.38,94094.0,34.053302
43,TX,1026860.59,18138800.58,19165661.17,1135043.0,16.885405
35,OH,1616581.07,18042852.11,19659433.18,1385100.0,14.193512
5,CO,2082879.02,17755216.77,19838095.79,1775831.0,11.171162
33,NC,951036.05,6028220.92,6979256.97,675469.0,10.332461
49,WI,936633.30,9089798.44,10026431.74,1020567.0,9.824374


### GROWTH IN CRAFT BREWERIES

In [33]:
sort_df = master_df
sort_df['% Change No. Breweries'] = ((sort_df['2015'] - sort_df['2011']) / sort_df['2011']) * 100
pct_change = sort_df[['State', '% Change No. Breweries']].sort_values(by='% Change No. Breweries', ascending=False)[:15]
pct_change

,State,% Change No. Breweries
34,ND,350.000000
3,AR,333.333333
0,AL,300.000000
24,MS,300.000000
9,FL,235.555556
43,TX,220.338983
35,OH,217.777778
46,VA,210.000000
23,MN,200.000000
29,NH,193.333333


Creating a calculated column in the DataFrame, we can look at the amount of change in the number of Craft Breweries between the years of 2011 and 2015. To note, the amounts above are percentages! One could conclude that these huge percentage changes are simply due to a state with a small number of breweries adding a few more, but the top third of the list contains states with large numbers already (FL, TX, OH, NY). In fact, the lowest % growth in the number of breweries is still a 1/3 increase in the number of craft breweries. Further more, 30 out of 50 states have doubled the number of craft breweries of more.  

In [34]:
# bar chart using Bokeh
bar3 = Bar(pct_change, values='% Change No. Breweries', label=['State'],
           title="Top States, % Change No. Breweries", legend=False, color='navy', plot_width=800)

bar3.title.text_font_size = '16pt'

show(row(bar3))

### UNEMPLOYMENT RATE  
Using the same methodology as comparing the percent growth in the number of breweries, we can look at the unemployment rate change of each state.  

In [35]:
sort_df = master_df
sort_df['% Change Unemp. Rate'] = ((sort_df['2015 Rate'] - sort_df['2011 Rate']) / sort_df['2011 Rate']) * 100
unemp_change = sort_df[['State', '% Change Unemp. Rate']].sort_values(by='% Change Unemp. Rate', ascending=False)

bar4 = Bar(unemp_change, values='% Change Unemp. Rate', label=['State'],
           title="% Change in Unemployment Rate", legend=False, color='#E74C3C', plot_width=900)

bar4.title.text_font_size = '16pt'

show(row(bar4))

The chart above shows the decrease in the unemployment rate for each state from 2011 to 2015. Not surprisingly, every state has seen a decrease in state-wide unemployment. Some of the states with high growth in craft breweries also appear here for larger decreases in state unemployment rate.  

The drop in unemployment rate is certainly not solely caused by everyone working at a craft brewery, but the craft brewery movement may have been driven by those out of work, with no other options. This may have perfectly coincided with a perfect storm of 18-35 year-olds preferring to spend money on experiences and technology allowing for easier access to smaller businesses. Many people after losing their source of income in 2008-2009, looked to use other talents or hobbies to make ends meet.  

It should be noted that many of the states with larger drops in unemployment also have other major industries, with the explosion in tech providing jobs in states like California, Colorado, Ohio, and New York.  

In [36]:
beer_cor = unemp_change.merge(pct_change, on='State')
beer_cor

scatter = Scatter(beer_cor, x='% Change Unemp. Rate', y='% Change No. Breweries', color='#1ABC9C', 
                  title="Change in Unemp. Rate vs. Craft Beer Growth (Top 15 States)", marker='State', 
                  legend="top_left", plot_height=650, plot_width=800)

scatter.title.text_font_size = '16pt'

scatter.legend.orientation = 'horizontal'

show(scatter)

The scatter plot shows a correlation between the growth in breweries and the decline in unemployment rate. Other factors are obviously at play, but the relationship is certainly interesting.  

## CONCLUSION  

This project merely touches the surface of a very large and diverse industry. I wanted to pull together data from a variety of sources and make some broad conclusions from it. More research into production volume, the exact definitions of craft versus other brewers, and other data would need to be combined to make a more in-depth study.  

From most of the information I have gathered, it appears that the growth of craft brewing is driven by societal changes (looking at the ratio or migration of age groups into certain states would be interesting), economic factors, and the spread of information (it's easier than ever to learn how to do almost anything because of the wealth of information available on-line).  

Mainly I have learned that beer is made and consumed in huge quantities, and that every state has some sort of beer production, with others leading the way over other states due to population size, culture, and laws on alcohol production.  